In [ ]:
import requests
import pandas as pd
import concurrent.futures
import time
import random
import re  # Import regex for strict ZIP code cleaning

# **🔹 Step 1: Set Headers (MUST INCLUDE YOUR EMAIL)**
USER_AGENTS = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.0.0 Safari/537.36",
]

EMAIL = "{}"  # Replace with your real email

session = requests.Session()
session.headers.update({
    "User-Agent": f"{random.choice(USER_AGENTS)} ({EMAIL})"
})

# **🔹 Step 2: Download SEC Company List**
def download_sec_company_list(max_retries=5):
    url = "https://www.sec.gov/files/company_tickers.json"
    
    for attempt in range(max_retries):
        try:
            response = session.get(url, timeout=10)
            if response.status_code == 200:
                company_data = response.json()
                df = pd.DataFrame.from_dict(company_data, orient='index')
                df["CIK"] = df["cik_str"].astype(str).str.zfill(10)  # Format CIK as 10 digits
                df = df.rename(columns={"title": "Company Name", "ticker": "Ticker"})
                
                # Remove duplicate company names (keep first)
                df = df.drop_duplicates(subset=["Company Name"], keep="first")

                print(f"✅ SEC company list loaded. {len(df)} unique companies found.")
                return df
            
            elif response.status_code == 403:
                wait_time = (attempt + 1) * 20  # **Exponential backoff**
                print(f"⚠️ 403 Forbidden. Retrying in {wait_time}s... (Attempt {attempt+1})")
                time.sleep(wait_time)

            else:
                print(f"❌ Failed to download SEC company list. Status Code: {response.status_code}")
                return pd.DataFrame()

        except requests.RequestException as e:
            print(f"❌ Error: {e}")
            time.sleep(10)

    return pd.DataFrame()  # **Return empty DataFrame after max retries**

# **🔹 Step 3: Strict ZIP Code Cleaning Function**
def clean_zip(zip_code):
    """ Keeps only numeric values and ensures it's a valid 5-digit ZIP code. """
    if zip_code and isinstance(zip_code, str):
        zip_code = re.sub(r'\D', '', zip_code)  # ✅ Remove non-numeric characters
        if len(zip_code) >= 5:
            return zip_code[:5]  # ✅ Keep only the first 5 digits
    return "Unknown"  # ✅ Return "Unknown" for missing or invalid ZIP codes

# **🔹 Step 4: Fetch ZIP Code with Cleaning**
def fetch_company_zip(cik, max_retries=3):
    url = f"https://data.sec.gov/submissions/CIK{cik}.json"
    
    for attempt in range(max_retries):
        try:
            response = session.get(url, timeout=5)
            if response.status_code == 200:
                data = response.json()

                business_zip = clean_zip(data.get("addresses", {}).get("business", {}).get("zipCode", "Unknown"))
                mailing_zip = clean_zip(data.get("addresses", {}).get("mailing", {}).get("zipCode", "Unknown"))

                time.sleep(random.uniform(0.5, 1.5))  # **Randomized wait time**
                return cik, business_zip, mailing_zip

            elif response.status_code == 429:  # **Rate-Limited (Too Many Requests)**
                wait_time = (attempt + 1) * 10
                print(f"⚠️ Rate limit hit. Retrying in {wait_time}s... (Attempt {attempt+1})")
                time.sleep(wait_time)

            elif response.status_code == 403:  # **Blocked**
                print(f"❌ 403 Forbidden for CIK {cik}. Skipping.")
                return cik, "Unknown", "Unknown"

            else:
                print(f"⚠️ No ZIP code for CIK {cik}. Status Code: {response.status_code}")

        except requests.RequestException as e:
            print(f"❌ Error fetching ZIP for CIK {cik}: {e}")
            time.sleep(5)

    return cik, "Unknown", "Unknown"  # **After max retries, return Unknown**

# **🔹 Step 5: Parallel Execution (All Companies)**
def collect_company_data():
    sec_company_df = download_sec_company_list()
    if sec_company_df.empty:
        return pd.DataFrame()

    cik_list = sec_company_df["CIK"].tolist()  # ✅ Get ALL companies
    results = []

    with concurrent.futures.ThreadPoolExecutor(max_workers=3) as executor:  # **Lower workers to avoid blocking**
        future_to_cik = {executor.submit(fetch_company_zip, cik): cik for cik in cik_list}
        for future in concurrent.futures.as_completed(future_to_cik):
            results.append(future.result())

    # Convert results into DataFrame
    df_final = pd.DataFrame(results, columns=["CIK", "Business ZIP", "Mailing ZIP"])
    df_final = sec_company_df.merge(df_final, on="CIK", how="left")  # Merge with company info

    return df_final

# **🔹 Step 6: Run and Save (All Companies)**
df_companies = collect_company_data()
output_path = "C:/Users/potat/OneDrive/Wooserk/02_data/SEC/02_data/sec_company_zipcodes.csv"
df_companies.to_csv(output_path, index=False)

print(f"✅ Data saved: {output_path}")
print(df_companies.head())